<a href="https://www.kaggle.com/code/asad11914/book-recommender-for-quiz?scriptVersionId=172333853" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Book Recommender for Quiz System

This notebook is demonstrate the book recommender system that used in Quiz platform. The recommendation is content-based, and based on how much a student is performing in particular subject. For example if student is prforming bad on biology, then the recommender system is returning lists of biology book. For now, its just recommending book, and can be expanded to recommending other learning material such as journal or blog.

Author : Muhammad Asaduddin

# 1. Load Data and Library

## Load Libraries
----
We use pandas library to processing the data easily, especially for csv files. we use Sklearn to turn string categories into number categories, and numpy to create numpy array

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

## Load Data

### Download Data
We use Book data from Indonesian's kemendikbudristek (Ministry of Education, Culture, Research and Technology), for both textbooks, and non-textbooks. We also need to download or define the lookup table for Id

In [2]:
!curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/text_books.csv
!curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/non_text_books.csv
# !curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/publishers.csv
# !curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/text_book_school_levels.csv
# !curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/text_book_school_classes.csv
# !curl -O https://siplah.kemdikbud.go.id/sds/lookup-tables/msts/books/text_book_subjects.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2026k  100 2026k    0     0   808k      0  0:00:02  0:00:02 --:--:--  809k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5276k  100 5276k    0     0  1917k      0  0:00:02  0:00:02 --:--:-- 1917k


In [3]:
book_df = pd.read_csv('./text_books.csv')
# subjects = pd.read_csv('./text_book_subjects.csv')
# classes = pd.read_csv('./text_book_school_classes.csv')
# publishers = pd.read_csv('./publishers.csv')

In [4]:
book_df

,id,isbn,nuib,title,author,edition,publication_year,description,subject_id,classification_id,class_id,school_level_id,publisher_id,prices,physical_description,synopsis,category_id,category_path,category_is_leaf
0,004acdf2-2e34-4d78-a132-33c5bda6561c,NaN,BTPL00001-000220-4126,Selalu Berhemat Energi 4B Teks Tematik Terpadu...,Dhiah Saptorini; Agus Wahyudi,NaN,NaN,NaN,935a14ff-1e68-4419-99b4-bdad47b2cb13,c8c14d0e-7bca-4c50-95c3-e6cc68c710f1,d109dcf2-60a6-42c9-9450-0a77baa0d79f,41a81c2a-166f-400d-b9a1-f6cf0c973e1f,054db149-8fce-40f9-b92e-a4810fa757b8,"[{""id"":""22631ca0-b021-4d5c-8d99-6a38bf5913a8"",...","{""size"":""A4"",""width"":21,""weight"":"""",""page_cont...",NaN,1618,"""Barang"",""Buku Teks"",""Buku Siswa"",""Jenjang SD/...",True
1,009074b7-a413-4183-8965-7ff242d018a3,NaN,BTPL00002-014020-0185,Buku Guru Tema 4 Keluargaku 1 Untuk SD/MI Kela...,Annisa Nur Rohmah; Istiana Nen Arienti; Slamet...,NaN,NaN,NaN,7784f0b2-2422-47be-aeef-76904e8e9631,c617a25f-f013-423d-a5f6-16be3a8d89d3,91d98fdf-e385-4102-8938-64af720718c9,41a81c2a-166f-400d-b9a1-f6cf0c973e1f,9157eb7e-9c1a-4b9d-aeed-727fd98fc9af,"[{""id"":""cc6bafb0-86f9-4dd3-b5dd-b30c94ee931d"",...","{""size"":""B5"",""width"":17.5,""weight"":"""",""page_co...",NaN,1760,"""Barang"",""Buku Teks"",""Buku Guru"",""Jenjang SD/M...",True
2,011f0232-f1ad-4aff-80b1-da70b2321825,NaN,BTPL00003-005120-8549,"Buku Siswa Pendidikan Jasmani, Olahraga, dan K...",Sardianto,NaN,NaN,NaN,7c35f50b-4f39-473b-9096-5f7e3474e3ff,c8c14d0e-7bca-4c50-95c3-e6cc68c710f1,e84c73cd-57c5-4442-b856-c7fdba0752df,6bdd711e-9a16-4449-bacf-a08f1879b36b,3c493ae1-f62e-41f4-9479-28b035715a10,"[{""id"":""415a467b-4b6e-465d-8edd-e80a78ad44a7"",...","{""size"":""B5"",""width"":17.5,""weight"":"""",""page_co...",NaN,1667,"""Barang"",""Buku Teks"",""Buku Siswa"",""Jenjang SMP...",True
3,013ba3b3-d7c6-4ea7-a9aa-f98778ef3bf6,978-602-244-696-5,BTPL00854-045823-3968,Buku Kurikulum Merdeka_Buku Panduan Guru Pendi...,"Susi Bonardy, Yenny Suria",1.0,2021.0,NaN,62233bc3-50e9-40b0-8221-d379e91c388d,c617a25f-f013-423d-a5f6-16be3a8d89d3,e4b14f52-715b-44d2-a82f-17e09dabef4f,41a81c2a-166f-400d-b9a1-f6cf0c973e1f,c1a471cf-53d9-483d-b958-d200724ef17e,"[{""id"":""98850c06-bf9e-4f0b-ab7f-c10ff97f1bdf"",...","{""size"":""17,6 x 25"",""width"":17,""weight"":"""",""pa...",NaN,1768,"""Barang"",""Buku Teks"",""Buku Guru"",""Jenjang SD/M...",True
4,01554a5e-9f62-435d-9a93-d8e099d11783,NaN,BTPL00005-045820-5298,Tema 7: Perkembangan Teknologi,NaN,NaN,2017.0,Untuk SD/MI Kelas 3,6145713e-b201-4cdb-b784-a7764abcad52,c8c14d0e-7bca-4c50-95c3-e6cc68c710f1,cc4b9872-a57f-4ca0-9945-70d794196fbc,41a81c2a-166f-400d-b9a1-f6cf0c973e1f,c1a471cf-53d9-483d-b958-d200724ef17e,"[{""id"":""a0518541-8c70-428e-ace4-f0074d262b73"",...","{""size"":""210 x 297"",""width"":"""",""weight"":"""",""pa...",NaN,1609,"""Barang"",""Buku Teks"",""Buku Siswa"",""Jenjang SD/...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,fee113d6-ee36-4d7d-b054-3a979c3cffe5,978-602-244-856-3,BTPL00831-045823-4293,Buku Kurikulum Merdeka_Buku Panduan Guru Sejar...,"Martina Safitry, Indah Wahyu Puji Utami, Zein ...",1.0,2021.0,NaN,7a60ed58-60f5-4bf0-a724-140f194e0493,c617a25f-f013-423d-a5f6-16be3a8d89d3,4807005f-e2cc-4ec1-8b27-4fbe09d72690,b5092d4e-c5a5-4f3d-89ac-70661366eac7,c1a471cf-53d9-483d-b958-d200724ef17e,"[{""id"":""98850c03-21df-46a8-84d7-b5c815b587fc"",...","{""size"":""17,6 x 25"",""width"":17,""weight"":"""",""pa...",NaN,1911,"""Barang"",""Buku Teks"",""Buku Guru"",""Jenjang SMA/...",True
1490,ff04fc84-db33-4ee0-ad1a-b06476dc6ccf,978-602-244-448-0,BTPL00770-045823-7906,Buku Kurikulum Merdeka_Pendidikan Pancasila da...,"Muhammad Sapei, Trezadigjaya, Prayogo",1.0,2021.0,NaN,9a653172-ed0d-4c03-99b5-2d88d4cadcaa,c8c14d0e-7bca-4c50-95c3-e6cc68c710f1,86ea3e63-6928-41cb-a75a-40852e2cca96,6bdd711e-9a16-4449-bacf-a08f1879b36b,c1a471cf-53d9-483d-b958-d200724ef17e,"[{""id"":""98850bf7-1e65-4b59-a941-3d84b8306762"",...","{""size"":""17,6 x 25"",""width"":17,""we

# 2. Cleaning The Data

## Create new Dataframe to just include important metadata
---- 
The Previous DataFrame contains all information about the book. It contains datas like author, publisher, year, price, and other. Because we recommend book based on quizes result, we don't need any metadata beside school level, class level, and subject. To make efficient data frame, we make another dataframe to just contain important data that we need for recommender. The reason we made a new data frame is that to make sure if we mess up the data frame, we can just go back without re-load the entire data from csv.


While investigate the data, we notice there is a column named `category_path` which contain all important data like subject, school level, and class level. We decide to use that column and turn that column into different columns

## Helper Function and Standarizer Dictionaries

Helper Function is help to accelerate preprocessing data. There are two helper function. First function, `preprocess_value` split data from `ctegory_path` and clear unecessary words. The second function, `remove_string_in_lsit` is used to remove class level and school level in category column

Standarizer dictionaries ( `class_map` and `school_list` ) are used to turn string category into numbers

In [5]:
# turn category_path values into different columns 
def preprocess_value(x: str) -> list:
    #repalce double ticks, 'Jenjang', and 'Kelas' into empty string
    x = x.replace('"', '')
    x = x.replace('Jenjang ', '')
    x = x.replace('Kelas ', '')
    #split string in comma sign
    x_list = x.split(',')
    return x_list

# remove specific string in the list. This used to get rid of class level on category column
def remove_string_in_list(x: str, remove_list: list[str]) -> str:
    # create list contain number from 00 to 10
    num_list = [ f'0{i}' if i <= 9 else f'{i}' for i in range(13)]
    # copy the list of string that we tryng to remove
    stopword = remove_list.copy()
    # extend the remove list to include list that contain numbers
    stopword.extend(num_list)
    # get rid of dash symbol, and pre and post spaces
    x = x.replace('-', '')
    x = x.strip()
    # split string into list, and begin to remove the string from the list
    x_list = x.split(' ')
    x_list = [word for word in x_list if word not in stopword]
    x_list = [subject_similar[word] if word in subject_similar.keys() else word for word in x_list]
    # join the list to build the entire subject
    x = ' '.join(x_list)
    x = x.strip()
    return x

In [6]:
class_map = {
    'PAUD' : 0.0,
    '1' : 1.0,
    '2' : 2.0,
    '3' : 3.0,
    '4' : 4.0,
    '5' : 5.0,
    '6' : 6.0,
    '7' : 7.0,
    '8' : 8.0,
    '9' : 9.0,
    '10': 10.0,
    '11': 11.0,
    '12': 12.0,
    'Semua Kelas' : 13.0
}

school_list = {
    'PAUD' : np.array([1,0,0,0,0,0]),
    'SD/MI' : np.array([0,1,0,0,0,0]),
    'SMP/MTs' : np.array([0,0,1,0,0,0]),
    'SMA/MA' : np.array([0,0,0,1,0,0]),
    'SMK/MAK' : np.array([0,0,0,0,1,0]),
    'SMA/SMK' : np.array([0,0,0,1,1,0]),
    'SMA/MA - SMK/MAK' : np.array([0,0,0,1,1,0]),
    'SDLB - SMPLB - SMA/KLB' : np.array([0,0,0,0,0,1]),
    'Umum' : np.array([0,1,1,1,1,1])
}

subject_similar = {
    'Pendidikan Agama Budha': 'Pendidikan Agama Buddha',
    'PJOK' : 'Pendidikan Jasmani Olahraga dan Kesehatan',
}

The steps to make new data frame to recommendation are:
1. Make helper function to split data from `catgory_path` column and a function to remove class level in subject
2. Include only the important columns from previous data frame
3. Apply helper function to clean data
4. Remove the books that for teacher and PAUD level
5. Standarize the similar subject name

In [7]:
np.array([1,0,1,2,4])

array([1, 0, 1, 2, 4])

In [8]:
book_df_clean = book_df.loc[:, ['id', 'title']]
columns = ['usage', 'school_level', 'class_level', 'subject']

book_df_clean[columns] = book_df['category_path'].apply(
    lambda x: pd.Series(preprocess_value(x)[2:])
)
school_level = list(school_list.keys())
book_df_clean['subject'] = book_df_clean.subject.map(lambda x: remove_string_in_list(x, school_level))
book_df_clean = book_df_clean[book_df_clean['usage'] != 'Buku Guru']
book_df_clean = book_df_clean[book_df_clean['class_level'] != 'PAUD'] 
book_df_clean.reset_index(drop=True, inplace=True)
book_df_clean.subject = book_df_clean.subject.apply(lambda x: subject_similar[x] if x in subject_similar.keys() else x)
# book_df_new = book_df_new.dropna()
# book_df_new = book_df_new.reset_index().loc[:,'id':'subject']

In [9]:
book_df_clean

,id,title,usage,school_level,class_level,subject
0,004acdf2-2e34-4d78-a132-33c5bda6561c,Selalu Berhemat Energi 4B Teks Tematik Terpadu...,Buku Siswa,SD/MI,4,Tema 2
1,011f0232-f1ad-4aff-80b1-da70b2321825,"Buku Siswa Pendidikan Jasmani, Olahraga, dan K...",Buku Siswa,SMP/MTs,7,Pendidikan Jasmani Olahraga dan Kesehatan
2,01554a5e-9f62-435d-9a93-d8e099d11783,Tema 7: Perkembangan Teknologi,Buku Siswa,SD/MI,3,Tema 7
3,01c7db56-2e52-4a30-8749-282a94e2b7ff,"Tema 7 Benda, Hewan, dan Tanaman di Sekitarku ...",Buku Siswa,SD/MI,1,Tema 7
4,01e4effa-ec64-49f1-b4de-939ae8754d80,Ilmu Pengetahuan Alam,Buku Siswa,SMP/MTs,7,IPA
...,...,...,...,...,...,...
770,fe4634a3-31d8-421e-b609-c261839ef705,"Tema 6 Lingkungan Bersih, Sehat, dan Asri Buku...",Buku Siswa,SD/MI,1,Tema 6
771,ff04fc84-db33-4ee0-ad1a-b06476dc6ccf,Buku Kurikulum Merdeka_Pendidikan Pancasila da...,Buku Siswa,SMP/MTs,8,Pendidikan Pancasila
772,ff1db59b-6036-475a-920a-c7cc95289e00,Bahasa Indonesia,Buku Siswa,SMA/MA,12,Bahasa Indonesia
773,ff43461c-3ddc-4931-9760-ed5b3be4c3c5,Pendidikan Agama Islam dan Budi Pekerti,Buku Siswa,SMP/MTs,7,Pendidikan Agama Islam


# 3. Create User Data

## Create user data via pandas data frame

In this section we are going to create user data to build our recommender system. For example only, we create user data using pandas dataframe. We define user id, quiz's school level, quiz's class level, quiz's suject, and score.

*note: On real world application, or in deployment, we use user's history of quizes that stored in database. In database, we create result table wich contain userId, quizId, and result. While the quiz's school level, quiz class level, and quiz subject is in different table called quizes that store entire quiz metadata*

In [10]:
user_data = pd.DataFrame([
    ['1234', 3, 11, 31, 90],
    ['3211', 1, 4, 31, 80],
    ['1234', 3, 11, 13, 60],
    ['1234', 3, 12, 6, 60],
    ['3211', 1, 7, 31, 20],
    ['1234', 5, 12, 31, 70],
    ['3211', 1, 5, 6, 60],
    ['1234', 6, 10, 13, 80],
    ['3211', 1, 4, 6, 80],
], columns=['user_id', 'school_level', 'class_level', 'subject', 'score'])

user_data

,user_id,school_level,class_level,subject,score
0,1234,3,11,31,90
1,3211,1,4,31,80
2,1234,3,11,13,60
3,1234,3,12,6,60
4,3211,1,7,31,20
5,1234,5,12,31,70
6,3211,1,5,6,60
7,1234,6,10,13,80
8,3211,1,4,6,80


# 4. Simple Content-Based Filtering

Our Objective is to give books recommendation based on users performaces on quizes. Thus, this objective cn be achieved in content-based filtering. One different from usual content-based filtering is the item to predict is quiz metadata, not another book.

Content-based filtering is similar to Decision Support System (DSS). First, we have `Items matrix` where items have certain number in each category. Then, we have `Users matrix` where Users have set `weight` for each `category` that called `preferences`. We then compute user's `preferences` to the item matrix, and rank it. The difference between DSS and Content-Based Filtering is the user's `preferences`  are computed based on past interaction. 

On this section, we compute the books matrix, and user matrix. We convert subject column in books dataframe, `book_df_clean` into one-hot, and convert `school_level` and `class_level` columns to number category. We get user's preferences based on quiz score. Where user preferences contructed by average school level and class level, and normalized summation quiz score of the same subject. 

The steps to make content-based recommendation are:
1. Create Book Matrix
2. Create User Matrix
3. Perform Calculation (such as dot product, euclidean distance) to make recommendation

## 1. Create Book Matrix

Create book matrix that contains id, class level, one-hot school level, and one-hot subject

In [11]:
book_matrix = book_df_clean.loc[:,['id', 'school_level', 'class_level']]

### Turn Subject to one-hot vector

In [12]:
enc = OneHotEncoder(sparse_output=False, dtype=int)
enc.fit(book_df_clean[['subject']])

columns = [j for items in enc.categories_ for j in items]
book_matrix[columns] = enc.transform(book_df_clean[['subject']])

In [13]:
#book_df_clean.drop(['school_level','class_level','subject'], axis=1, inplace=True)
# enc.inverse_transform([[1, 45]])
school_columns = ['PAUD', 'SD', 'SMP', 'SMA', 'SMK', 'SLB']
book_matrix[school_columns] = book_matrix.school_level.apply(
    lambda x: pd.Series(school_list[x])
)
book_matrix.class_level = book_df_clean.class_level.map(class_map)
book_matrix.drop(['school_level'], axis=1, inplace=True)

In [14]:
book_matrix

,id,class_level,Agribisnis Perikanan,Agribisnis Tanaman,Agribisnis Ternak,Agriteknologi Pengolahan Hasil Pertanian,Akuntansi dan Keuangan Lembaga,Animasi,Bahasa Indonesia,Bahasa Inggris,...,Tema 8,Tema 9,Usaha Layanan Pariwisata,Usaha Pertanian Terpadu,PAUD,SD,SMP,SMA,SMK,SLB
0,004acdf2-2e34-4d78-a132-33c5bda6561c,4.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,011f0232-f1ad-4aff-80b1-da70b2321825,7.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,01554a5e-9f62-435d-9a93-d8e099d11783,3.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,01c7db56-2e52-4a30-8749-282a94e2b7ff,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,01e4effa-ec64-49f1-b4de-939ae8754d80,7.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,fe4634a3-31d8-421e-b609-c261839ef705,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
771,ff04fc84-db33-4ee0-ad1a-b06476dc6ccf,8.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
772,ff1db59b-6036-475a-920a-c7cc95289e00,12.0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
773,ff43461c-3ddc-4931-9760-ed5b3be4c3c5,7.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## 2. Create User Matrix

Create a new User Matrix that have same columns as the book matrix

1. Filter who we want to give recommendation by filtering `user_id column`
2. Create new matrix in shape 1 x 84 (1 class_level + 77 subject + 6 school_level)
3. Map users score and `school_level` to one-hot
4. Normalize subjects and school levelscolumns

In [15]:
user_selected = user_data[user_data['user_id'] == '1234']
user_selected
user_matrix = pd.DataFrame(np.zeros((1,84), dtype=np.float32))

user_matrix[[0]] = user_selected.loc[:,'class_level'].mean()
# user_matrix[1] = user_data.loc[user_data['user_id'] == '1234', 'class_level'].mean()
for i in range(77):
    user_matrix[i+1] = 60 / user_selected.loc[(user_data['subject'] == i), 'score'].sum()

user_matrix.iloc[:,78:] = user_selected.school_level.apply(
    lambda x : pd.Series(list(school_list.values())[x])
).sum()


user_matrix.loc[:,78:] = user_matrix.iloc[:,78:].div(user_matrix.iloc[:,78:].sum(axis=1), axis=0)

user_matrix.replace([np.inf, -np.inf], 0, inplace=True)
user_matrix.loc[:,1:78] = user_matrix.iloc[:,1:78].div(
                            user_matrix.iloc[:,1:78].sum(axis=1), 
                            axis=0)
user_matrix.fillna(0, inplace = True)
#user_matrix.iloc[:,33]

/tmp/ipykernel_18/751210274.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  user_matrix[i+1] = 60 / user_selected.loc[(user_data['subject'] == i), 'score'].sum()
/tmp/ipykernel_18/751210274.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  user_matrix[i+1] = 60 / user_selected.loc[(user_data['subject'] == i), 'score'].sum()
/tmp/ipykernel_18/751210274.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  user_matrix[i+1] = 60 / user_selected.loc[(user_data['subject'] == i), 'score'].sum()
/tmp/ipykernel_18/751210274.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  user_matrix[i+1] = 60 / user_selected.loc[(user_data['subject'] == i), 'score'].sum()
/tmp/ipykernel_18/751210274.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  user_matrix[i+1] = 60 / user_selected.loc[(user_data['subject'] == i), 'score'].sum()
/tmp/ipykernel_18/751210274.py:8: RuntimeWarning: divide by zero encountered in scalar div

In [16]:
user_matrix[[0,1,7,14,32,78, 79, 80, 81, 82, 83]]

,0,1,7,14,32,78,79,80,81,82,83
0,11.2,0.0,0.554455,0.237624,0.207921,0.0,0.0,0.0,0.714286,0.285714,0.0


## Get Recommendation

1. Compute Distances between user and each books
2. Sort distances from shortest to longest
3. limit result (30 row)
4. Get the books data

In [17]:
distances = np.sum(np.square(book_matrix.iloc[:,1:].to_numpy() - user_matrix.to_numpy()), axis=1)
#distances.sort_values(0, ascending=False)[:20]
recommendations = pd.DataFrame(distances).sort_values(0)[:30]
# np.argmax(distances)

In [18]:
book_df_clean.iloc[recommendations.index]

,id,title,usage,school_level,class_level,subject
695,e1ea0fde-9ca2-4c4c-a4d2-a85f7649666c,Buku Kurikulum Merdeka_Bahasa Indonesia Tingka...,Buku Siswa,SMA/MA,11,Bahasa Indonesia
187,604e4327-3fba-4843-a78a-bc7558fb4c8b,Bahasa Indonesia,Buku Siswa,SMA/MA,11,Bahasa Indonesia
645,cee83570-2e3d-40e3-aad9-12b0068864a2,Buku Kurikulum Merdeka_Cerdas Cergas Berbahasa...,Buku Siswa,SMA/SMK,11,Bahasa Indonesia
772,ff1db59b-6036-475a-920a-c7cc95289e00,Bahasa Indonesia,Buku Siswa,SMA/MA,12,Bahasa Indonesia
242,74b0c8ac-bf34-4b33-8a38-c94dc4b54a7a,Bahasa Indonesia,Buku Siswa,SMA/MA,12,Bahasa Indonesia
318,987af67a-dc74-44ed-b634-a6a41e8b60cf,Buku Kurikulum Merdeka_Biologi untuk SMA/MA Ke...,Buku Siswa,SMA/MA,11,IPA
319,987af67a-ed53-4ff0-a78f-946f119f9e8e,Buku Kurikulum Merdeka_Kimia untuk SMA/MA Kela...,Buku Siswa,SMA/MA,11,IPA
320,987af67b-0115-4b35-9413-024b93eeb1a6,Buku Kurikulum Merdeka_Fisika untuk SMA/MA Kel...,Buku Siswa,SMA/MA,11,IPA
765,fcc1f60a-7eb0-4693-df0f-7c754e834a2e,Buku Kurikulum Merdeka_Cerdas Cergas Berbahasa...,Buku Siswa,SMA/MA - SMK/MAK,12,Bahasa Indonesia
41,1337f799-bd85-f3d5-6465-0b4973ee29b4,Buku Kurikulum Merdeka_Bahasa Indonesia Tingka...,Buku Siswa,SMA/MA - SMK/MAK,12,Bahasa Indonesia
